In [ ]:
from selenium import webdriver
import os
from bs4 import BeautifulSoup
import pandas as pd
import time
import csv
import urllib.request as ur

In [ ]:
def TV_chosun(keyword):
    #크롬 드라이버 가져오기
    browser=webdriver.Chrome('chromedriver.exe')
    #TV_조선 홈페이지 들어가서 키워드 검색하기
    browser.get('http://search.tvchosun.com/tvchosun/news.search?query={0}&starget=&scomplex=or&sdate=&categoryname=&categoryd2=&pageno=1&orderby=news&skind=&requery={1}&cont5=&category={2}&kind=&cont4=tab&sline=Y'.format(keyword,keyword,'뉴스'))
    
    #기사 목록에서 url 가져와서 list형태로 만들기
    url=[]
    html=browser.page_source
    soup=BeautifulSoup(html,'html.parser')
    attr=soup.select('div.detail > p.article_tit > a')
    for urls in attr:
        url.append(urls['href'])
    browser.find_element_by_xpath('//*[@id="wrap"]/div[2]/div[3]/span/a[1]').click()
    html=browser.page_source
    soup=BeautifulSoup(html,'html.parser')
    attr=soup.select('div.detail > p.article_tit > a')
    for urls in attr:
        url.append(urls['href'])
    browser.find_element_by_xpath('//*[@id="wrap"]/div[2]/div[3]/span/a[2]').click()
    html=browser.page_source
    soup=BeautifulSoup(html,'html.parser')
    attr=soup.select('div.detail > p.article_tit > a')
    for urls in attr:
        url.append(urls['href'])
    browser.find_element_by_xpath('//*[@id="wrap"]/div[2]/div[3]/span/a[3]').click()
    html=browser.page_source
    soup=BeautifulSoup(html,'html.parser')
    attr=soup.select('div.detail > p.article_tit > a')
    for urls in attr:
        url.append(urls['href'])
    
    #url list에 들어있는 데이터를 하나씩 가져와서 기사 제목과 본문을 추출 후 dictionary로 저장하기
    dict={}
    for i in range(len(url)):
        browser.get(url[i])
        html=browser.page_source
        soup=BeautifulSoup(html,'html.parser')
        title=soup.find('h3',{'class':'title'}).text
        title=title.replace('\r','').replace('\n','').replace('\t','')
        text=soup.find('div',{'class':'article_detail_body'}).text
        text=text.replace('\n','').replace('\t','').replace('\r','')
        dict[title]=text
    
    #dictionary 파일들을 csv파일로 저장
    with open('TV조선_{0}.csv'.format(keyword),'w',-1,encoding='utf-8') as f:
        w=csv.writer(f)
        w.writerow(dict.keys())
        w.writerow(dict.values()) 
    
    #수정을 위해 csv파일 읽어오기
    chosun=pd.read_csv('C:/Users/dsp12/Downloads/TV조선_{0}.csv'.format(keyword),encoding='utf-8')
    
    #행렬전환
    chosun=chosun.transpose()
    list_b=list
    title=list_b(chosun.index)
    list_c=list
    text=chosun[0]
    text=list_c(text)
    source=pd.DataFrame({'Title':title,'Text':text})
    #60개의 기사들만 뽑아낸다.
    source=source.iloc[:60]
    #Press이름의 열을 만들어 언론사 이름을 저장시킨다.
    source['Press']='TV조선'
    source=source[['Press','Title','Text']]
    source.to_csv('TV조선_{0}.csv'.format(keyword),encoding='utf-8-sig')